# Tumor Classification Project

### Description:

The purpose of this project is the employ computer vision and a CNN in order to classify MRI images of brains with and without tumors. The primary purpose is to train the model on a binary classifier (tumor or no tumor), then secondarily to classify the type of tumor if one is present.

### Dataset:
https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection

The data set provides the MRI images in their own folders of either 'yes' or 'no' (referring to if the scan contains a tumor). There are 154 images in the yes folder (tumor) and 91 in the no folder (healthy).

### Importing Packages

In [ ]:
#Importing primary packages
import numpy as np
import torch
import glob
import cv2

#Importing Dependencies
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score

Matplotlib is building the font cache; this may take a moment.


### Data Reading

In [ ]:
#First creating empty lists
tumor = []
healthy = []

#Then iterating over all images and appending them to the empty lists
for f in glob.iglob('C:/Data/brain_tumor_dataset/yes/*.jpg'):
    img = cv2.imread(f)
    img = cv2.resize(img, (128,128))
    b, g, r = cv2.split(img)
    img = cv2.merge([r,g,b])
    tumor.append(img)

for f in glob.iglob('C:/Data/brain_tumor_dataset/no/*.jpg'):
    img = cv2.imread(f)
    img = cv2.resize(img, (128,128))
    b, g, r = cv2.split(img)
    img = cv2.merge([r,g,b])
    healthy.append(img)

In [ ]:
#Verifying all the previous steps worked properly; we should see 154 images, sized to 128x128 dimensions and 3 different channels for rgb values.
tumor.shape

In [14]:
#Making the lists into arrays that can be processed by the ML model
healthy = np.array(healthy)
tumor = np.array(tumor)

#Combining the two datasets so we can randomize, split, and feed to the model.
All = np.concatenate((healthy,tumor))

In [15]:
#Checking that our concatenation was done properly
All.shape

(245, 128, 128, 3)